In [1]:
from __future__ import annotations

from dataclasses import dataclass
from typing import Dict, Optional, Tuple
import pandas as pd

In [2]:
# Load price data (same CSV as Task 1, but reloaded here)
from google.colab import drive

drive.mount('/content/drive')

CSV_PATH = "/content/drive/MyDrive/data/Nat_Gas.csv"
df = pd.read_csv(CSV_PATH, parse_dates=[0], index_col=0)
df.index.name = "Dates"



# Build the price curve: index = Dates, values = Prices
price_curve = df["Prices"].sort_index()

Mounted at /content/drive


/tmp/ipython-input-3248267387.py:7: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = pd.read_csv(CSV_PATH, parse_dates=[0], index_col=0)


In [3]:
@dataclass(frozen=True)
class StoragePricingInputs:
    # Price curve: index = dates (pd.Timestamp), values = price ($/MMBtu)
    price_curve: pd.Series

    # Schedules (volumes in MMBtu)
    injections: Dict[pd.Timestamp, float]   # {date: injected_volume}
    withdrawals: Dict[pd.Timestamp, float]  # {date: withdrawn_volume}

    # Constraints
    injection_rate_per_day: float           # MMBtu/day
    withdrawal_rate_per_day: float          # MMBtu/day
    max_storage_volume: float               # MMBtu

    # Storage costs (choose either or both)
    storage_cost_per_day_fixed: float = 0.0     # $/day (fixed rent)
    storage_cost_per_mmbtu_day: float = 0.0     # $/(MMBtu*day) (inventory-dependent)

    # Initial inventory (MMBtu)
    initial_inventory: float = 0.0


def price_storage_contract(
    x: StoragePricingInputs,
    return_ledger: bool = True
) -> Tuple[float, Optional[pd.DataFrame]]:
    """
    Storage contract value (USD) = sum over days of:
      - trade cashflows (buy/sell on injection/withdrawal dates using price_curve)
      - storage cashflows (fixed per day + variable per inventory-day)

    Assumptions:
      - interest rates = 0 (no discounting)
      - no transport delay
      - no holiday/weekend adjustments

    Returns:
      (total_value_usd, ledger_df or None)
    """

    # --- basic validations ---
    if x.injection_rate_per_day < 0 or x.withdrawal_rate_per_day < 0:
        raise ValueError("Rates must be >= 0.")
    if x.max_storage_volume < 0:
        raise ValueError("max_storage_volume must be >= 0.")
    if x.initial_inventory < 0 or x.initial_inventory > x.max_storage_volume:
        raise ValueError("initial_inventory must be within [0, max_storage_volume].")

    if not isinstance(x.price_curve, pd.Series) or x.price_curve.empty:
        raise ValueError("price_curve must be a non-empty pd.Series.")
    if not isinstance(x.price_curve.index, pd.DatetimeIndex):
        raise ValueError("price_curve.index must be a pd.DatetimeIndex.")

    # Normalize schedule keys to pd.Timestamp
    injections = {pd.Timestamp(k): float(v) for k, v in x.injections.items()}
    withdrawals = {pd.Timestamp(k): float(v) for k, v in x.withdrawals.items()}

    if any(v < 0 for v in injections.values()) or any(v < 0 for v in withdrawals.values()):
        raise ValueError("Injected/withdrawn volumes must be >= 0.")

    # Timeline = daily range from first event to last event
    event_dates = sorted(set(injections.keys()) | set(withdrawals.keys()))
    if not event_dates:
        return 0.0, None

    start, end = min(event_dates), max(event_dates)
    timeline = pd.date_range(start=start, end=end, freq="D")

    price_curve = x.price_curve.sort_index()

    inventory = x.initial_inventory
    total_value = 0.0
    rows = []

    for d in timeline:
        inj = injections.get(d, 0.0)
        wd = withdrawals.get(d, 0.0)

        # Rate constraints
        if inj > x.injection_rate_per_day + 1e-12:
            raise ValueError(f"Injection exceeds rate on {d.date()}: {inj} > {x.injection_rate_per_day}")
        if wd > x.withdrawal_rate_per_day + 1e-12:
            raise ValueError(f"Withdrawal exceeds rate on {d.date()}: {wd} > {x.withdrawal_rate_per_day}")

        # Trade cashflows: buy on injection dates, sell on withdrawal dates
        trade_cf = 0.0
        if inj > 0.0:
            if d not in price_curve.index:
                raise KeyError(f"Missing price for injection date {d.date()}")
            p = float(price_curve.loc[d])
            trade_cf -= inj * p

        if wd > 0.0:
            if d not in price_curve.index:
                raise KeyError(f"Missing price for withdrawal date {d.date()}")
            p = float(price_curve.loc[d])
            trade_cf += wd * p

        # Update inventory (same-day execution)
        inv_before = inventory
        inventory = inventory + inj - wd

        # Storage volume constraints
        if inventory < -1e-9:
            raise ValueError(f"Inventory goes negative on {d.date()} (before={inv_before}, inj={inj}, wd={wd}).")
        if inventory > x.max_storage_volume + 1e-9:
            raise ValueError(f"Capacity exceeded on {d.date()} (inv={inventory} > max={x.max_storage_volume}).")

        # Storage costs for the day (post-trade end-of-day inventory)
        storage_cf = -(x.storage_cost_per_day_fixed + x.storage_cost_per_mmbtu_day * inventory)

        day_cf = trade_cf + storage_cf
        total_value += day_cf

        rows.append({
            "date": d,
            "price": float(price_curve.loc[d]) if d in price_curve.index else float("nan"),
            "inject_mmbtu": inj,
            "withdraw_mmbtu": wd,
            "inventory_end_mmbtu": inventory,
            "cashflow_trade_usd": trade_cf,
            "cashflow_storage_usd": storage_cf,
            "cashflow_total_usd": day_cf,
            "value_cum_usd": total_value,
        })

    ledger = pd.DataFrame(rows).set_index("date")
    return total_value, (ledger if return_ledger else None)

In [4]:
#Test 1
import pandas as pd

dates = pd.date_range("2025-01-01", "2025-01-10", freq="D")
prices = pd.Series([2.0, 2.1, 2.0, 2.2, 2.3, 2.6, 2.8, 3.0, 3.1, 3.2], index=dates)

x = StoragePricingInputs(
    price_curve=prices,
    injections={pd.Timestamp("2025-01-02"): 50_000, pd.Timestamp("2025-01-03"): 50_000},
    withdrawals={pd.Timestamp("2025-01-09"): 60_000, pd.Timestamp("2025-01-10"): 40_000},
    injection_rate_per_day=60_000,
    withdrawal_rate_per_day=70_000,
    max_storage_volume=120_000,
    storage_cost_per_day_fixed=1_000,
    storage_cost_per_mmbtu_day=0.0,
    initial_inventory=0.0
)

value, ledger = price_storage_contract(x, return_ledger=True)
print("Contract value (USD):", value)
print(ledger[["inject_mmbtu","withdraw_mmbtu","inventory_end_mmbtu","cashflow_total_usd","value_cum_usd"]])


Contract value (USD): 100000.0
            inject_mmbtu  withdraw_mmbtu  inventory_end_mmbtu  \
date                                                            
2025-01-02       50000.0             0.0              50000.0   
2025-01-03       50000.0             0.0             100000.0   
2025-01-04           0.0             0.0             100000.0   
2025-01-05           0.0             0.0             100000.0   
2025-01-06           0.0             0.0             100000.0   
2025-01-07           0.0             0.0             100000.0   
2025-01-08           0.0             0.0             100000.0   
2025-01-09           0.0         60000.0              40000.0   
2025-01-10           0.0         40000.0                  0.0   

            cashflow_total_usd  value_cum_usd  
date                                           
2025-01-02           -106000.0      -106000.0  
2025-01-03           -101000.0      -207000.0  
2025-01-04             -1000.0      -208000.0  
2025-01-05   

In [6]:
#Test 2
x_bad = StoragePricingInputs(
    price_curve=prices,
    injections={pd.Timestamp("2025-01-02"): 100_000, pd.Timestamp("2025-01-03"): 100_000},
    withdrawals={},
    injection_rate_per_day=200_000,
    withdrawal_rate_per_day=200_000,
    max_storage_volume=250_000,   # increased
    storage_cost_per_day_fixed=0.0
)


# Expected: ValueError (capacity exceeded)
value, ledger = price_storage_contract(x_bad)


In [7]:
#Test 3
x_var = StoragePricingInputs(
    price_curve=prices,
    injections={pd.Timestamp("2025-01-02"): 100_000},
    withdrawals={pd.Timestamp("2025-01-10"): 100_000},
    injection_rate_per_day=200_000,
    withdrawal_rate_per_day=200_000,
    max_storage_volume=200_000,
    storage_cost_per_day_fixed=0.0,
    storage_cost_per_mmbtu_day=0.0005
)

value, ledger = price_storage_contract(x_var, return_ledger=True)
print("Contract value (USD):", value)
print("Total storage cost (USD):", ledger["cashflow_storage_usd"].sum())


Contract value (USD): 109600.0
Total storage cost (USD): -400.0
